In [37]:
import cv2
import pandas as pd
import time
from pathlib import Path
import numpy as np

In [38]:
folder_path = r'\\cimec-storage5\acn_lab\shared_acn\Matilde\zebrafish\social_preference'
video_name = 'fish1_hab_socialpreference_2023-11-07T17_51_17'
video_path = folder_path + fr"\{video_name}" + '.avi'

In [39]:
# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)  # replace 'your_video.mp4' with your video file

# Get the video properties
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create a window and set the callback for mouse events
# callback function for mouse events is a function that gets executed 
# when a mouse-related action occurs within a specified window

# Set the desired playback speed (e.g., 15fps)
desired_fps = 90
frame_delay = int(1000 / desired_fps)

# Initialize variables
paused = False

frames = np.arange(0, total_frames)
position_fish = ['False'] * len(frames)
fish_data = {'frame': frames, 'fish_inside_area': position_fish}
inside_area = False  # Initial state

In [40]:
# Function to draw a line and collect points
def draw_line(event, x, y, flags, param):
    global points_line1
    if event == cv2.EVENT_LBUTTONDOWN:
        points_line1.append((x, y))
        if len(points_line1) > 1:
            cv2.line(img, points_line1[-2], points_line1[-1], (0, 0, 255), 2)  # Draw line1 in red
            cv2.imshow('Draw Lines', img)

In [41]:
# Open the video capture object
cap = cv2.VideoCapture(video_path)  # replace 'your_video.mp4' with your video file
ret, img = cap.read()

# Create a window and set the callback for mouse events
cv2.namedWindow('Draw Lines')
points_line1 = []
cv2.setMouseCallback('Draw Lines', draw_line)

# Draw the first line (line1) on the first frame
while True:
    cv2.imshow('Draw Lines', img)
    key = cv2.waitKey(0)
    if key == 27:# Press 'Esc' key or draw at least 2 points to exit
        break
    elif key == ord('c'):
        points_line1 = []
        img = cap.read()[1]
        print(key)
        
cv2.destroyAllWindows()

In [46]:
#Draw coordinates for line 2

midpoint_line1 = ((points_line1[0][0] + points_line1[1][0]) // 2, (points_line1[0][1] + points_line1[1][1]) // 2) # Calculate the midpoint of line1
slope_line1 = (points_line1[1][1] - points_line1[0][1]) / (points_line1[1][0] - points_line1[0][0]) # Calculate the slope of line1
slope_line2 = -1 / slope_line1 # Calculate the perpendicular slope for line2
b_line2 = midpoint_line1[1] - slope_line2 * midpoint_line1[0] # Calculate the y-intercept for line2 using the midpoint

# Calculate the coordinates for line2 based on the frame size
height, width, _ = img.shape
x1_line2 = int(midpoint_line1[0] - width / 2)  # Adjust the length of line2 as needed
y1_line2 = int(slope_line2 * x1_line2 + b_line2)

x2_line2 = int(midpoint_line1[0] + width / 2)  # Adjust the length of line2 as needed
y2_line2 = int(slope_line2 * x2_line2 + b_line2)

points_line2 = [(x1_line2, y1_line2), (x2_line2, y2_line2)]

In [47]:
# Function to handle mouse events for setting inside_area
def set_inside_area(event, x, y, flags, param):
    global inside_area
    if event == cv2.EVENT_LBUTTONDOWN:
        inside_area = True
    elif event == cv2.EVENT_RBUTTONDOWN:
        inside_area = False

In [48]:
cv2.namedWindow('Video')
cv2.setMouseCallback('Video', set_inside_area)
cv2.resizeWindow('Video', 200, 150)  # Set the size of the window (width, height)

while True:
    if not paused:
        ret, frame = cap.read()
        if not ret:
            break

    cv2.line(frame, points_line2[0], points_line2[1], (0, 255, 0), 2)  # Draw line2 in green

    cv2.imshow('Video', frame)

    key = cv2.waitKey(frame_delay) & 0xFF
    if key == 27:
        break
        
    elif key == 32:
        paused = not paused
        
    elif key == ord('6'): 
        paused = False
        
    elif key == ord('4'):  
        # Go back 5 frames (adjust as needed)
        cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, cap.get(cv2.CAP_PROP_POS_FRAMES) - 60))
        
        
    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))    
    fish_data['fish_inside_area'][frame_number] = inside_area
        
    print(frame_number, inside_area)

# Save data to CSV
#df = pd.DataFrame(data)


#df.to_csv('fish_activity_data.csv', index=False)

# Release the VideoCapture and close the window
cap.release()
cv2.destroyAllWindows()